In [1]:
import os
import sys
from collections import defaultdict
import csv
import pandas as pd

### This code preprocesses already created entity names in NERecog.ipynb and creates lists of merged entities and documents. Then replaces the multi word phrases by joining them with "_". 
### This then outputs entity_vocab, entity_corpus and entity_dictionary

In [2]:
#Build complete vocabulary

vocab = []

with open("/home/rin/CrossPlatform/Code/Entity_Vocab/Only_PROPN/Gab_vocab.txt", 'r') as gfile:
    for line in gfile:
        word = str(line.strip())
        if len(word) > 4:
            vocab.append(word)
        
with open("/home/rin/CrossPlatform/Code/Entity_Vocab/Only_PROPN/Twitter_vocab.txt", 'r') as gfile:
    for line in gfile:
        word = str(line.strip())
        if len(word) > 4:
            vocab.append(word)

        
with open("/home/rin/CrossPlatform/Code/Entity_Vocab/Only_PROPN/Facebook_vocab.txt", 'r') as gfile:
    for line in gfile:
        word = str(line.strip())
        if len(word) > 4:
            vocab.append(word)


In [3]:
print len(vocab)

313


In [4]:
vocab = list(set(vocab))
print len(vocab)

271


In [5]:
import pickle

with open('/home/rin/CrossPlatform/Code/entity_vocab_onlyPROPN', 'wb') as pfile:
    pickle.dump(vocab, pfile)


In [6]:
#create document corpus

#paths
# Twitter path
tp = '/home/rin/CrossPlatform/raw_text/Twitter/'

#Facebook path
fp = '/home/rin/CrossPlatform/raw_text/Facebook/'

#Gab path
gp = '/home/rin/CrossPlatform/raw_text/Gab/'

extremists = ["FrankGaffney.txt", "LarryPratt.txt", "PaulRamsey.txt", "StefanMolynuex.txt", "JosephFarah.txt", "MikeCernovich.txt", "PeterBrimelow.txt" ]
ideologies = ["antiMuslim", "antiGov", "whiteNat", "altRight", "antiGov", "maleSupr", "whiteNat"]

#documents
documents = []
folders = [tp, fp, gp]

for fold in folders:
    for extreme in extremists:
        with open(fold+extreme, 'r') as rfile:
            for line in rfile:
                if len(line) > 5:
                    documents.append(line)


In [7]:
print len(documents)

81637


In [8]:
#Join phrases in the vocab
x = "terms and conditions"
print x.split(" ")
print "_".join(x.split(" "))

['terms', 'and', 'conditions']
terms_and_conditions


In [9]:
#create dictionary for phrases and joined phrases
translate_vocab = defaultdict()
for v in vocab:
    translate_vocab[v] = "_".join(v.split(" "))

In [10]:
import re
from text_cleaner import remove, keep

from text_cleaner.processor.common import ASCII
from text_cleaner.processor.chinese import CHINESE, CHINESE_SYMBOLS_AND_PUNCTUATION
from text_cleaner.processor.misc import RESTRICT_URL

def cleaning2(text):
    text = remove(text, [RESTRICT_URL])
    text = re.sub(r'\b(?:(?:https?|ftp)://)?\w[\w-]*(?:\.[\w-]+)+\S*(?<![.,])', ' ', text.lower())
    words = re.findall(r'[a-z.,]+', text)
    
    return ' '.join(words)

In [11]:
for i in range(len(documents)):
    test_str = cleaning2(documents[i])
    for v in vocab:
        if v in test_str:
            test_str = test_str.replace(v, translate_vocab[v])
            
    documents[i] = test_str

In [12]:
with open("/home/rin/CrossPlatform/Code/entity_corpus_onlyPROPN", "wb") as dfile:
    pickle.dump(documents, dfile)
    

In [13]:
with open("/home/rin/CrossPlatform/Code/entity_vocab_onlyPROPN", 'r') as vfile:
    vcb = pickle.load(vfile)

In [14]:
#dump dictionaries also
with open('/home/rin/CrossPlatform/Code/entity_dict_onlyPROPN', 'wb') as handle:
    pickle.dump(translate_vocab, handle, protocol=pickle.HIGHEST_PROTOCOL)